In [1]:
import pandas as pd
import numpy as np

# 1

In your own words, describe what vector embeddings are and what they are useful for.

Vector embeddings are transformations or representations of objects to lists of numbers that allow us to perform vector operations. They are useful to calculate semantic similarity as proximity in a vector space, allowing tasks in clustering, recommendation and classification.

# 2

What do you think is the best distance criterion to estimate how far two embeddings (vectors) are from each other? Why?

There are multiple distance metrics that can be used in order to achieve certain objetives and nature of data. 

**Euclidean distance:** It's commonly used and intuitive, useful to calculate "straight" distances between vectors.

**Manhattan Distance:**  It's useful when there are certain dimensions more importan than others, because it calculates the sum of the absolute differences between corresponding elements of the vectors.

**Minkowski Distance:** It allows to use both euclidean and manhattan allowing to adjust emphasis on different dimensions.

**Cosine:** It allows to calculate similarity by in direction rather than magnitude, allowing to work with different sizes vectors as in text simmilarity.

**Mahalanobis Distance:** It takes into account the covariance structure of the data, making it suitable for cases where the dimensions are correlated. It is particularly useful in multivariate analysis and anomaly detection.

**Jaccard Distance:** It's commonly used for comparing sets or binary data. It is frequently used in text analysis and recommendation systems.

**Hamming Distance:** This metric is used for comparing binary vectors of equal length. It counts the number of positions at which the corresponding elements of the vectors differ. It's commonly used in error-correcting codes and genetics.



# 3

Let us build a Q&A (question answering) system! 😀For this, consider the following steps

Pick whatever text you like, in the order of 20+ paragraphs.

In [19]:
data_file

<_io.TextIOWrapper name='D:/OneDrive - Universidad de Antioquia/Analitica y Ciencia de Datos/2/Machine learning 2/Taller3/theblackcat.txt' mode='r' encoding='UTF-8'>

In [14]:
with open("D:/OneDrive - Universidad de Antioquia/Analitica y Ciencia de Datos/2/Machine learning 2/Taller3/theblackcat.txt",'r') as data_file:
    for line in data_file:
        data = line.split(r'\n')
        print(data)

['FOR the most wild, yet most homely narrative which I am about to pen, I neither expect nor solicit belief. Mad indeed would I be to expect it, in a case where my very senses reject their own evidence. Yet, mad am I not -- and very surely do I not dream. But to-morrow I die, and to-day I would unburthen my soul. My immediate purpose is to place before the world, plainly, succinctly, and without comment, a series of mere household events. In their consequences, these events have terrified -- have tortured -- have destroyed me. Yet I will not attempt to expound them. To me, they have presented little but Horror -- to many they will seem less terrible than barroques. Hereafter, perhaps, some intellect may be found which will reduce my phantasm to the common-place -- some intellect more calm, more logical, and far less excitable than my own, which will perceive, in the circumstances I detail with awe, nothing more than an ordinary succession of very natural causes and effects.\n']
['From 

In [15]:
!pip install -U sentence-transformers

     ---------------------------------------- 86.0/86.0 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 7.6/7.6 MB 5.7 MB/s eta 0:00:00
     -------------------------------------- 172.3/172.3 MB 3.7 MB/s eta 0:00:00
     ---------------------------------------- 1.2/1.2 MB 6.3 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 8.0 MB/s eta 0:00:00
     -------------------------------------- 977.5/977.5 kB 6.2 MB/s eta 0:00:00
     -------------------------------------- 294.8/294.8 kB 6.1 MB/s eta 0:00:00
     -------------------------------------- 173.4/173.4 kB 1.7 MB/s eta 0:00:00
     ---------------------------------------- 2.1/2.1 MB 7.8 MB/s eta 0:00:00
     ---------------------------------------- 5.7/5.7 MB 6.8 MB/s eta 0:00:00
     ------------------------------------- 268.3/268.3 kB 16.1 MB/s eta 0:00:00
     -----------------------

In [17]:
pip install transformers


Note: you may need to restart the kernel to use updated packages.


In [93]:
from transformers import AutoModel, AutoTokenizer
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

model_name = "sentence-transformers/all-MiniLM-L6-v2"
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load your text from the TXT file and split it into paragraphs/chunks
with open("D:/OneDrive - Universidad de Antioquia/Analitica y Ciencia de Datos/2/Machine learning 2/Taller3/theblackcat.txt", "r") as file:
    text = file.read()

paragraphs = text.split("\n")  # You may need to adjust the delimiter based on your text format


# Define a list of user questions
user_questions = ["What did he do to the cat","Is the cat alive"]

# Initialize an empty list to store results
results = []

# Encode the user questions and paragraphs, and calculate cosine similarity
for question in user_questions:
    question_encoding = tokenizer(question, padding=True, truncation=True, return_tensors="pt")
    question_outputs = model(**question_encoding)

    question_embedding = question_outputs.last_hidden_state.mean(dim=1).detach().numpy()  # Use mean pooling for the question

    paragraph_similarities = []
    for paragraph in paragraphs:
        paragraph_encoding = tokenizer(paragraph, padding=True, truncation=True, return_tensors="pt")
        paragraph_outputs = model(**paragraph_encoding)

        paragraph_embedding = paragraph_outputs.last_hidden_state.mean(dim=1).detach().numpy()  # Use mean pooling for the paragraph

        # Calculate cosine similarity between the question and paragraph embeddings
        similarity = cosine_similarity(np.reshape(question_embedding, (1, -1)), np.reshape(paragraph_embedding, (1, -1)))
        paragraph_similarities.append(similarity.item())

    # Sort paragraphs by similarity
    sorted_paragraphs = [paragraph for _, paragraph in sorted(enumerate(paragraphs), key=lambda x: paragraph_similarities[x[0]], reverse=True)]

    # Store the top N chunks for this question
    N = 5  # Number of top chunks to return
    results.append((question, sorted_paragraphs[:N]))

# Print or use the results as needed
for question, top_chunks in results:
    print(f"Question: {question}")
    print("Top Chunks:")
    for i, chunk in enumerate(top_chunks, 1):
        print(f"{i}. {chunk}")
    print()


Question: What did he do to the cat
Top Chunks:
1. With my aversion to this cat, however, its partiality for myself seemed to increase. It followed my footsteps with a pertinacity which it would be difficult to make the reader comprehend. Whenever I sat, it would crouch beneath my chair, or spring upon my knees, covering me with its loathsome caresses. If I arose to walk it would get between my feet and thus nearly throw me down, or, fastening its long and sharp claws in my dress, clamber, in this manner, to my breast. At such times, although I longed to destroy it with a blow, I was yet withheld from so doing, partly by a memory of my former crime, but chiefly -- let me confess it at once -- by absolute dread of the beast.
2. Although I thus readily accounted to my reason, if not altogether to my conscience, for the startling fact just detailed, it did not the less fail to make a deep impression upon my fancy. For months I could not rid myself of the phantasm of the cat; and, during t

Results don't make so much sense, surely because text isn't as simple and it's hard for the model to make relations.

# 4

What do you think that could make these types of systems more robust in terms of semantics and functionality?

Maybe a bigger and diverse set of training could make those models more robust, also participation of experts in language in developing models, using feedback of users and collaborative research strategies